# U.K. Smart Meter Energy Report
## Author: Alexander Van Roijen
## Due Date: 12/10/2019

## I. Introduction
This notebook and repository is dedicated to a reproducible investigation into a multi year study on energy consumption in the United Kingdom (U.K.). I will first provide background into the problem the data was collected for, related work, and the motivation behind this analysis. I will then explicitly describe the data and the permissions surrounding it.Afterwards, research questions will be described and the methods used to answer them. Finally the results will be presented showing how dynamic time of use (ToU) tariffs on residential electric demand in the U.K. exercise an extremely significant decrease in net electric consumption compared to their static pricing counterparts. In addition, I will highlight the annual savings per household per year under these ToU tariffs. I will also provide results demonstrating the use of Sparse Identification of Nonlinear Dynamics (SINDy) to predict future demand response. References as well as all code used to generate the data for analysis can be found at the end of the notebook.

## II. Background
According to the world bank as of 2017, we are at a global electrification of 88.8%, meaning about 88.8% of the worlds population has access to electricity $^{[1]}$. At the same time, it is well known that the global population has been increasing dramatically, and furthermore that urbanization has been observed over the past century. In fact, urbanization is at many countries that have had electricity for a long time are learning how to better manage its generation and distribution. Here are some useful terms to keep in mind:
- System balancing (SB): The matching of energy supply to the time at which energy is being requested 
- Constraint management (CM): Limited capacity to transport electricity to all locations (bottleneck effect) means we need to manage this as best we can.
- Demand response (DR): Altering consumer electric usage by providing financial incentives.

Consequently, many utilities and governments are working together to implement demand response programs to help with system balancing and constraint management. In particular, a popular form is through Dynamic time-of-use (dToU) tariffs, as opposed to standard static pricing methods. The main idea being, we charge customers more during high demand periods for energy to discourage wasteful or inefficient uses to times of lower demand. By using these tariffs, one can hope to  achieve demand response to help with both system balancing and constraint management. This notebook will explore exactly how well this method worked. But first, why am I interested in the first place?
### Motivation
I have always been fascinated by energy consumption, starting back to the [energy dis-aggregation task](https://web.stanford.edu/group/peec/cgi-bin/docs/events/2011/becc/presentations/3%20Disaggregation%20The%20Holy%20Grail%20-%20Carrie%20Armel.pdf) that I helped do some research in during [my undergrad](https://aaai.org/ojs/index.php/AAAI/article/view/3872). It would be interesting to try similar work on this data. However, this notebook focuses on a simple analyses on how dynamic pricing impacts DR.
### Related Work
There has been significant investigation already into demand response in various countries across the world. with a particular focus on European countries $^{[2,3,4]}$. Furthermore, despite low usage of this dataset within the first few years of its release, it has seen a large increase in usage to study various subjects such as demand prediction $^{[5]}$. However, since its release was mainly aiming to understand if dToU tariffs were impacting DR in the way we wanted them too, very little analysis has been done. In particular, I will be building off some of the work done by Gareth Piggot $^{[6]}$

So what does our data look like?

## III. Data
Smart meter readings from the United Kingdom over a three year period (2012 to 2014) at half hour intervals. What this means is that from ~5500 households we have total energy draw at 30 minute intervals. They were "recruited as a balanced sample representative of the Greater London population"$^{[1]}$. There are two types of groups, those under dynamic time of use (ToU) pricing and others under static pricing(Std). The main objective of this data collection process was to understand how users of various demographics respond to dynamic pricing compared to static pricing. "The signals given were designed to be representative of the types of signal that may be used in the future to manage both high renewable generation ... and also test the potential to use high price signals to reduce stress on local distribution grids during periods of stress" $^{[1]}$

Of the 5500 households included in the study, 1100 houses opted into the ToU pricing practice. while the remaining 4400

### Data templates
#### raw data 
| Column | type | description |
|--------|------|-------------|
| index(country)  | string  | the country name is the index for this csv          |
|   page     |    string  |     the title of the political article in question        |
|     rev_id   |   int  |      revision id that uniquely identifies the wikipedia articles most recent submission       |
| article_quality | string | quality of this page determined by ORES |
| population | float | the population of the country associated with the politician of the page in million |
| region | string | the region associated with this country |

#### myAnalysesData

### Licensing & Permissions
The terms and conditions from the london data store explicitly state that anyone "May use the data contained in this site for any purpose, providing it does not infringe the terms and conditions."$^{[2]}$. However, I will also take this time to note __"...the Greater London Authority cannot warrant the quality or accuracy of the data"__$^{[2]}$

With this taken care of, lets dive into what we will be exploring in the rest of this notebook

## IV. Research Questions

### 1) Are ToU customers responding to this pricing? In particular, do they consume more or less than their static pricing counterparts? Is this relationship significant?

Are we achieving forms of SB or CM using this set of tariffs? Can we see through the noise that this relationship is actually true?

### 2) If this relationship is significant, what kind of savings do we get when switching to ToU? 

Even if they are significant, what kind of savings can we expect? Particularly, we want to know if the monetary cost of implementing the new system, along with its bureaucratic over head, offer significant gains.

### 3) Can we model future consumption using only past consumption as a signal?

In particular, I will be using Sparse Identification of Non-linear Dynamics(SINDy). This method allows us to hopefully find an equation that can model consumption over time in an interpretable and intuitive manner,

## V. Methods

For questions 1 and 2, I will 
- First, parse the data into the forms indicated above
- Second, use a groupby operation with pandas to determine average and standard deviation for both std and ToU groups for each thirty minute interval.
- Second, create an error plot to highlight the difference in consumption per half hour for both std and ToU groups.
- Third, run an unequal variances T-Test for each 30 minute interval to determine significance (or lack thereof).
- Fourth, take the difference between each interval, sum the up, and multiply over 365 days of the year to get annual savings for household kWh.

For question 3, I will
- First, create a function to window my data such that I can request it to create inputs of an inputted time delay.
    - More explicitly, createWindow(data,numHouses,windowSize) will return 

## VI. Results & Discussion

### Research Question 1
Are ToU customers responding to this pricing? In particular, do they consume more or less than their static pricing counterparts? Is this relationship significant?

!["errorplot for std and tou pricing usage over 30 min intervals"](images/stdvstouAll.jpg "erroplot for std and tou pricing usage over 30 min intervals")


The answer is yes to all of the above! If you want to see the technical details on how this graph was generated and the significance determined, please look at [this notebook](https://www.google.com)

### Research Question 2

### Research Question 3

!["SINDy results with lambda = 0.0001 and max iterations with 2 hour window or 4 time steps"](images/4dwindowRes.jpg "SINDy results with lambda = 0.0001 and max iterations with 2 hour window or 4 time steps")


## VII. Conclusion


### Things to keep in mind
- 1) Did those who opt into the survey begin as mindful people? This is a potential or were they not? We can test this! look at starting points of those who opted in, did they start conscious of consumption or not!

## References
- [] Access to electricity (% of population) (2019).  Retrieved November 22, 2019, from World B ank. https://data.worldbank.org/indicator/eg.elc.accs.zs. CC 4.0 License. 
- [] Department of Economic and Social Affairs. (2019). World urbanization prospects : the 2018 revision. New York: United Nations.
- [] Torriti, J., Hassan, M. G., & Leach, M. (2010). Demand response experience in Europe: Policies, programmes and implementation. Energy, 35(4), 1575-1583.
- [] Siano, Pierluigi. "Demand response and smart grids—A survey." Renewable and sustainable energy reviews 30 (2014): 461-478.
- [] Jordehi, A. R. (2019). Optimisation of demand response in electric power systems, a review. Renewable and Sustainable Energy Reviews, 103, 308-319.
- [] Sun, M., Wang, Y., Strbac, G., & Kang, C. (2018). Probabilistic peak load estimation in smart cities using smart meter data. IEEE Transactions on Industrial Electronics, 66(2), 1608-1618.
- [] Piggott, G (June 24th, 2015). Electricity Consumption in a Sample of London Households. Retrieved from https://data.london.gov.uk/blog/electricity-consumption-in-a-sample-of-london-households/
- [] UK Power Networks. (2015). SmartMeter Energy Consumption Data in London Households [Zip, Data File]. Retrieved from https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households
- [] Terms and Conditions. (2019). Retrieved November 22, 2019, from London Data Store, London Data Store Terms and Conditions. Website, https://data.london.gov.uk/about/terms-and-conditions/
- [] Jenkins, N., Long, C., & Wu, J. (2015). An overview of the smart grid in Great Britain. Engineering, 1(4), 413-421.
- [] Qdr, Q. J. U. D. E. (2006). Benefits of demand response in electricity markets and recommendations for achieving them. US Dept. Energy, Washington, DC, USA, Tech. Rep.
- [] Schofield, J. R., Carmichael, R., Tindemans, S., Bilton, M., Woolf, M., & Strbac, G. (2015). Low Carbon London project: Data from the dynamic time-of-use electricity pricing trial, 2013. [data collection]. UK Data Service. 
- [] Dutta, G. & Mitra, K. J Oper Res Soc (2017) 68: 1131. https://doi.org/10.1057/s41274-016-0149-4
- [] Schofield, J. (2015). Dynamic time-of-use electricity pricing for residential demand response: Design and analysis of the Low Carbon London smart-metering trial.

 

## Data Cleaning / Supporting Code